In [2]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [55]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [56]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [57]:
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
returns = merge(returns, log1p.(data_source[:T_Bill])./4)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
term = data_source[:Yeld_10Y] ./4 .- data_source[:T_Bill] ./4
dy = data_source[:Dividend_Yeld] #log.(data_source[:Dividend_Yeld])
data = merge(real_returns, dy, term, cpi)

291×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1952-03-01 │      1.98335e-5 │     0.0107337 │  0.00772445 │     0.0602359 │ ⋯
│ 1952-06-01 │       0.0581411 │    0.00263202 │ -0.00708528 │      0.057815 │ ⋯
│ 1952-09-01 │      -0.0185027 │    -0.0173351 │  0.00423886 │     0.0592197 │ ⋯
│ 1952-12-01 │       0.0932838 │     0.0220972 │   0.0089235 │     0.0538579 │ ⋯
│ 1953-03-01 │      -0.0437093 │   -0.00972202 │  0.00497517 │     0.0571967 │ ⋯
│ 1953-06-01 │      -0.0100139 │    -0.0219485 │ -0.00227055 │     0.0584603 │ ⋯
│ 1953-09-01 │     -0.00574595 │     0.0298874 │ -0.00299956 │     0.0596579 │ ⋯
│ 1953-12-01 │        0.078226 │     0.0248173 │  0.00767892 │     0.0572141 │ ⋯
│     ⋮      │        ⋮        │ 

In [6]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [7]:
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [8]:
returns_summarystats(to(from(returns, start),end_d), 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.144099,0.782737,0.0963943,0.261158,-1.01471,-0.845921,-0.0423317,0.0953652,1.77521,0.668946,0.142769
Price_10Y,0.0832445,0.569087,0.044665,0.156403,-0.289949,0.371142,-0.0740101,0.0442422,0.221941,0.536551,0.0280389
T_Bill,0.0101058,0.0582689,0.0227848,0.045069,9.9995e-5,0.292382,0.00149888,0.968883,-1.50635,2.25464,0.0172994


In [9]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.0717569,0.193813,0.0178775,0.0610642,-0.238141,-0.699168,-0.0145302,0.0849045,1.2878,0.24914,0.0283261
Price_10Y_CPI,0.0448954,0.167664,0.0049452,0.0347394,-0.0960374,0.432308,-0.0258376,0.0666902,0.789709,0.110149,0.00203425
T_Bill_CPI,0.00893438,0.025467,-0.000524846,0.00528326,-0.026825,-0.318287,-0.00503521,0.302165,0.492507,-0.0587446,0.000505661
Dividend_Yeld,0.00435984,0.0323637,0.0189393,0.0207695,0.0112582,0.765491,0.0164025,0.918403,0.717749,4.34404,0.0185857
Yeld_10Y_T_Bill,0.00293746,0.0095,0.00388508,0.006225,-0.0034,-0.236784,0.0017125,0.874579,-0.53084,1.3226,0.00395
CPI,0.00774615,0.0279226,0.00622104,0.00985738,-0.025392,-0.34528,0.00268687,0.0727647,2.47992,0.803114,0.00575376


In [10]:
returns_summarystats(s_data, 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.143514,0.775253,0.0715101,0.244257,-0.952565,-0.699168,-0.0581207,0.0849045,1.2878,0.498281,0.113304
Price_10Y_CPI,0.0897907,0.670655,0.0197808,0.138958,-0.38415,0.432308,-0.10335,0.0666902,0.789709,0.220299,0.00813698
T_Bill_CPI,0.0178688,0.101868,-0.00209938,0.021133,-0.1073,-0.318287,-0.0201408,0.302165,0.492507,-0.117489,0.00202265
Dividend_Yeld,0.00871968,0.129455,0.0757573,0.0830779,0.0450328,0.765491,0.06561,0.918403,0.717749,8.68808,0.0743426
Yeld_10Y_T_Bill,0.00587492,0.038,0.0155403,0.0249,-0.0136,-0.236784,0.00685,0.874579,-0.53084,2.6452,0.0158
CPI,0.0154923,0.11169,0.0248842,0.0394295,-0.101568,-0.34528,0.0107475,0.0727647,2.47992,1.60623,0.0230151


In [47]:
#returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
returns3y = percentchange(collapse(data_source[:Total_Stock, :Price_10Y],year, last), :log)
returns_summarystats(returns3y,3)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.262448,1.14822,0.3116,0.633328,-1.31799,-0.904378,0.0348067,-0.0653805,1.1152,1.18728,0.39273
Price_10Y,0.149818,0.997588,0.15512,0.30155,-0.535906,0.495645,0.00831092,-0.0530983,0.978126,1.03539,0.102748


In [11]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI,std
Total_Stock_CPI,-0.0295,0.1152,0.0788,-0.8405,4.0279,-5.652,-1.5289,0.07
Price_10Y_CPI,-0.0077,-0.1285,-0.0358,2.8038,-0.488,3.675,1.779,0.0427
T_Bill_CPI,-0.0122,-0.0249,-0.0146,1.0719,0.3155,0.119,1.0222,0.0073
Dividend_Yeld,0.0012,-0.003,-0.0016,0.002,0.9069,0.0919,0.0168,0.0015
Yeld_10Y_T_Bill,0.0003,0.0017,0.0066,-0.0515,0.0091,0.887,-0.0329,0.0012
CPI,0.0118,0.0262,0.0089,-0.0735,-0.3003,-0.0871,-0.0139,0.0075


,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI
Total_Stock_CPI,1.0,-0.16,0.03,-0.91,0.02,-0.01
Price_10Y_CPI,-0.16,1.0,0.44,0.23,-0.58,-0.49
T_Bill_CPI,0.03,0.44,1.0,0.16,-0.14,-0.99
Dividend_Yeld,-0.91,0.23,0.16,1.0,-0.07,-0.17
Yeld_10Y_T_Bill,0.02,-0.58,-0.14,-0.07,1.0,0.07
CPI,-0.01,-0.49,-0.99,-0.17,0.07,1.0


In [12]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_CPI coefficients"

Quantiles
       parameters       2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol    Float64   Float64   Float64   Float64   Float64 

            const    -0.0976   -0.0523   -0.0285   -0.0057    0.0397
  Total_Stock_CPI    -0.0689    0.0487    0.1144    0.1785    0.3026
    Price_10Y_CPI    -0.2660   -0.0401    0.0784    0.1923    0.4276
       T_Bill_CPI    -3.9014   -1.9048   -0.8643    0.2137    2.2030
    Dividend_Yeld     0.6261    2.8615    3.9956    5.1446    7.3632
  Yeld_10Y_T_Bill   -11.2454   -7.5941   -5.6556   -3.6177    0.2533
              CPI    -5.0192   -2.7842   -1.5505   -0.3574    2.0022


"Price_10Y_CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0487   -0.0221   -0.0079    0.0069    0.0350
  Total_Stock_CPI   -0.2418   -0.1669   -0.1282   -0.0897   -0.0172
    Price_10Y_CPI   -0.2465   -0.1077   -0.0352    0.0344    0.1780
       T_Bill_CPI    0.9280    2.1791    2.8188    3.4577    4.6800
    Dividend_Yeld   -2.5277   -1.1973   -0.5053    0.1838    1.5915
  Yeld_10Y_T_Bill    0.1213    2.4640    3.6467    4.9225    7.2741
              CPI   -0.3205    1.0656    1.7866    2.5065    3.9650


"T_Bill_CPI coefficients"

...

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0009   -0.0001    0.0003    0.0007    0.0015
  Total_Stock_CPI   -0.0016    0.0006    0.0017    0.0028    0.0049
    Price_10Y_CPI    0.0005    0.0045    0.0066    0.0086    0.0126
       T_Bill_CPI   -0.1056   -0.0697   -0.0515   -0.0331    0.0016
    Dividend_Yeld   -0.0492   -0.0107    0.0094    0.0289    0.0685
  Yeld_10Y_T_Bill    0.7840    0.8514    0.8867    0.9219    0.9902
              CPI   -0.0950   -0.0543   -0.0331   -0.0118    0.0281


"CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const    0.0043    0.0092    0.0118    0.0143    0.0191
  Total_Stock_CPI    0.0061    0.0192    0.0261    0.0328    0.0464
    Price_10Y_CPI   -0.0283   -0.0042    0.0088    0.0214    0.0470
       T_Bill_CPI   -0.4077   -0.1843   -0.0751    0.0369    0.2519
    Dividend_Yeld   -0.6625   -0.4207   -0.2988   -0.1750    0.0529
  Yeld_10Y_T_Bill   -0.7218   -0.3043   -0.0866    0.1258    0.5546
              CPI   -0.3921   -0.1409   -0.0138    0.1142    0.3643


"correlation matrix"

Quantiles
                       parameters      2.5%     25.0%     50.0%     75.0%      ⋯
                           Symbol   Float64   Float64   Float64   Float64   Fl ⋯

  Total_Stock_CPI_Total_Stock_CPI    0.0633    0.0686    0.0718    0.0751    0 ⋯
    Total_Stock_CPI_Price_10Y_CPI   -0.3299   -0.2187   -0.1567   -0.0929    0 ⋯
       Total_Stock_CPI_T_Bill_CPI   -0.1579   -0.0347    0.0309    0.0927    0 ⋯
    Total_Stock_CPI_Dividend_Yeld   -0.9383   -0.9223   -0.9122   -0.9007   -0 ⋯
  Total_Stock_CPI_Yeld_10Y_T_Bill   -0.1657   -0.0436    0.0199    0.0829    0 ⋯
              Total_Stock_CPI_CPI   -0.1862   -0.0703   -0.0084    0.0566    0 ⋯
    Price_10Y_CPI_Total_Stock_CPI   -0.3299   -0.2187   -0.1567   -0.0929    0 ⋯
      Price_10Y_CPI_Price_10Y_CPI    0.0386    0.0419    0.0438    0.0459    0 ⋯
         Price_10Y_CPI_T_Bill_CPI    0.2760    0.3848    0.4380    0.4889    0 ⋯
      Price_10Y_CPI_Dividend_Yeld    0.0435    0.1670    0.2286    0.2875    0 ⋯
    Price_10Y_CPI

In [13]:
vmodel.Y[end,:]

6-element Vector{Float64}:
  0.12159459085777868
  0.06515539231011935
  0.010346584402051042
  0.014635185560818083
 -0.0034000000000000002
  0.002421733143949112

In [48]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel,60)

for n in 1:6
    s_ann = annualise(scenarios[n,:,2:end],4)
    print_percentiles(s_ann,[0.05, 0.25, 0.5, 0.75, 0.95], string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0934175,-0.15864,-0.178376,-0.188372,-0.194005,-0.198061,-0.191255,-0.198262,-0.193051,-0.190673,-0.20113,-0.190341,-0.192514,-0.196614,-0.191602
0.25,0.0397755,-0.0159778,-0.033426,-0.0460369,-0.0527076,-0.0538032,-0.0492668,-0.0525277,-0.0451374,-0.0495567,-0.0515635,-0.0455981,-0.0461058,-0.0476197,-0.0461903
0.5,0.131719,0.080003,0.0635761,0.0551075,0.0491764,0.0469319,0.0506603,0.0508109,0.0535487,0.0526406,0.0490141,0.0534183,0.0531109,0.0556204,0.0551187
0.75,0.219515,0.180062,0.162367,0.158272,0.152598,0.151641,0.152878,0.152464,0.152652,0.154713,0.152256,0.156375,0.153629,0.157033,0.154195
0.95,0.358887,0.321794,0.309589,0.307338,0.301273,0.299227,0.29894,0.296485,0.299409,0.299576,0.300992,0.302135,0.298857,0.3028,0.297713


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.152132,-0.133468,-0.132336,-0.134883,-0.136841,-0.136752,-0.13857,-0.139363,-0.14027,-0.139473,-0.139999,-0.142536,-0.141223,-0.144245,-0.142691
0.25,-0.0748574,-0.0514382,-0.0487321,-0.0481211,-0.0494374,-0.0496532,-0.0507276,-0.0527508,-0.0516496,-0.0513245,-0.0536315,-0.0536981,-0.0561088,-0.056698,-0.0550984
0.5,-0.0195032,0.00677003,0.0108515,0.0106568,0.0114113,0.0144505,0.0106659,0.00900129,0.0115307,0.0097291,0.00958437,0.00682474,0.00721225,0.00503366,0.0071138
0.75,0.0358983,0.0642503,0.0690035,0.0710628,0.0730456,0.074975,0.0716477,0.0706267,0.0712538,0.0713638,0.0692352,0.0692849,0.0694742,0.0662652,0.0673738
0.95,0.113949,0.14847,0.154981,0.158443,0.16201,0.161746,0.159642,0.157718,0.161298,0.160286,0.157442,0.156141,0.158626,0.155102,0.156342


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0124958,-0.0157663,-0.0206852,-0.0252023,-0.02907,-0.0320216,-0.0340147,-0.035927,-0.0369016,-0.0382855,-0.0401812,-0.0407473,-0.0411566,-0.0420193,-0.0423194
0.25,0.00244337,0.00112163,-0.00191685,-0.00572809,-0.00803978,-0.0103002,-0.0116556,-0.0136125,-0.0145625,-0.0157128,-0.0170822,-0.0176464,-0.0186954,-0.0184127,-0.019055
0.5,0.012453,0.0132842,0.0105608,0.00829206,0.00633573,0.00497954,0.00312496,0.00188626,0.0011934,-0.00023026,-0.00121809,-0.00191264,-0.00251435,-0.00256619,-0.00287004
0.75,0.0224319,0.0251744,0.0234756,0.0225731,0.0206653,0.019771,0.0180429,0.0174688,0.017057,0.0162294,0.0146406,0.0139333,0.0134854,0.0133122,0.0130995
0.95,0.0373539,0.0423235,0.0416196,0.0412856,0.0418172,0.0406191,0.0399849,0.0399495,0.0395931,0.0388745,0.0370154,0.0368685,0.0365618,0.0366954,0.0361964


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.0417511,0.0332254,0.0307595,0.0301279,0.0303258,0.0314857,0.0324654,0.031952,0.0329403,0.0342727,0.0350932,0.0356494,0.0363542,0.036307,0.0369902
0.25,0.0491408,0.0448381,0.0436028,0.0438687,0.04473,0.0459587,0.0469269,0.0479596,0.0490432,0.0499138,0.0509931,0.0514646,0.0521298,0.0528021,0.0537804
0.5,0.0541253,0.0525793,0.0527613,0.0534481,0.0548321,0.0562653,0.0574265,0.05888,0.0600134,0.0608041,0.0616602,0.0626941,0.063051,0.0640796,0.0648716
0.75,0.0592827,0.0602811,0.0615956,0.0629378,0.0650216,0.0667715,0.0682749,0.0696521,0.0708686,0.0718745,0.0729804,0.0738743,0.0745083,0.0749842,0.0756832
0.95,0.0665382,0.0713222,0.0741399,0.0764104,0.0788159,0.0814821,0.0833736,0.0849037,0.0862929,0.0872324,0.0889717,0.0903484,0.0912425,0.0914875,0.091939


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0180069,-0.019539,-0.019481,-0.0188103,-0.0173135,-0.0168209,-0.0159413,-0.0153104,-0.014678,-0.014429,-0.0137051,-0.0130375,-0.0130344,-0.0122214,-0.0121666
0.25,-0.0126088,-0.011401,-0.0098207,-0.00803566,-0.00651826,-0.00546286,-0.00456804,-0.00350657,-0.00270814,-0.00187511,-0.00129878,-0.000739973,4.4311e-6,0.000253724,0.000474625
0.5,-0.0088352,-0.00561403,-0.00313599,-0.000938088,0.00099201,0.00252068,0.00364745,0.00474184,0.00589164,0.00647103,0.00743456,0.00809304,0.00852886,0.0090917,0.00944942
0.75,-0.00505972,0.000282134,0.00360668,0.00652504,0.00859009,0.0103003,0.0117482,0.0132306,0.0139995,0.0151525,0.0160719,0.0166724,0.0171522,0.0176498,0.0181434
0.95,0.000499839,0.00853407,0.0137575,0.017033,0.0198523,0.0221878,0.0237558,0.0253963,0.0263966,0.0274215,0.0280955,0.0292966,0.0294428,0.0302578,0.0305341


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.00985878,0.00568569,0.00534443,0.00442991,0.00392462,0.0030006,0.00309072,0.00260041,0.00179191,0.00180352,0.00253659,0.00131969,0.00121868,0.0010881,0.00118199
0.25,0.0252193,0.0216666,0.0215239,0.0208045,0.0198297,0.0193401,0.0193357,0.0189135,0.0180454,0.0178928,0.0181802,0.0175027,0.0176963,0.0173963,0.016811
0.5,0.0359743,0.0326121,0.0322769,0.0318077,0.0313052,0.0310621,0.0303533,0.0300746,0.0295121,0.0293792,0.0291715,0.0290731,0.0289651,0.0282409,0.0283072
0.75,0.0468309,0.0434881,0.0431313,0.0430068,0.0427309,0.0418836,0.0416658,0.0414846,0.0406047,0.0402991,0.0404679,0.0401164,0.0399802,0.039843,0.0396548
0.95,0.0623654,0.0597379,0.05932,0.0589128,0.0590726,0.0585584,0.0584343,0.057907,0.0569417,0.0566563,0.0567048,0.056754,0.0565563,0.0561381,0.0557528


In [25]:
scenarios_b = simulate(bmodel,60, 10)
for n in 1:6
    s_ann = annualise(scenarios_b[n,:,:],4)
    print_percentiles(s_ann,[0.02, 0.25, 0.5, 0.75, 0.98], string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0164852,-0.482036,-0.585131,-0.630199,-0.65057,-0.667927,-0.688733,-0.697536,-0.709056,-0.722662,-0.740901,-0.75988,-0.76692,-0.776805,-0.8015
0.25,0.162446,-0.0722056,-0.111875,-0.126213,-0.13447,-0.137593,-0.141874,-0.143344,-0.142986,-0.145602,-0.147624,-0.149344,-0.151635,-0.151461,-0.152935
0.5,0.224026,0.0797717,0.0675319,0.0647153,0.0613893,0.0613508,0.0601045,0.0639357,0.0648276,0.0633084,0.0636855,0.0650361,0.0653242,0.0639349,0.0652395
0.75,0.292144,0.239281,0.249309,0.253949,0.260752,0.264376,0.268988,0.272039,0.275484,0.278278,0.280194,0.284348,0.28524,0.286881,0.289622
0.98,0.476112,0.682707,0.753002,0.779514,0.802921,0.820636,0.852279,0.871937,0.89538,0.919205,0.961725,0.983947,1.01062,1.03502,1.06682


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.122657,-0.473723,-0.559674,-0.598516,-0.624504,-0.640631,-0.653512,-0.668172,-0.67833,-0.682136,-0.687101,-0.701168,-0.710851,-0.715499,-0.731081
0.25,0.00183517,-0.125069,-0.149547,-0.164506,-0.175764,-0.183383,-0.188648,-0.19402,-0.197773,-0.201762,-0.205741,-0.208889,-0.210466,-0.213195,-0.214535
0.5,0.049256,0.00774168,0.0113524,0.0109409,0.00895262,0.00869339,0.00901476,0.00659612,0.00492239,0.00423338,0.00191487,0.00258212,0.00124335,0.000703434,0.000251564
0.75,0.0929802,0.136624,0.172809,0.188323,0.195498,0.200398,0.203158,0.207547,0.208782,0.209721,0.209987,0.211131,0.212744,0.213286,0.215749
0.98,0.198708,0.47259,0.57831,0.620716,0.642014,0.660491,0.675681,0.678091,0.678528,0.681811,0.696524,0.701375,0.701774,0.70999,0.712244


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0334678,-0.125872,-0.173581,-0.206117,-0.234731,-0.25923,-0.281569,-0.29836,-0.317022,-0.334889,-0.348922,-0.368494,-0.386821,-0.404643,-0.42407
0.25,0.0024969,-0.0296544,-0.045751,-0.0568102,-0.0660593,-0.072716,-0.0778585,-0.0822786,-0.0860246,-0.0889832,-0.0913763,-0.0936775,-0.0956993,-0.0968766,-0.0981973
0.5,0.0189541,0.0131788,0.0107673,0.00890981,0.00639776,0.00595217,0.00485545,0.003728,0.00322252,0.00211359,0.00148076,0.00186077,0.00150115,0.00114263,0.00135534
0.75,0.0352632,0.0561196,0.0677952,0.0748645,0.0802122,0.0842968,0.0877692,0.0909174,0.0934749,0.0958135,0.0969084,0.0984,0.100089,0.100804,0.101991
0.98,0.0719413,0.155352,0.198465,0.229997,0.253875,0.277427,0.299879,0.320772,0.339773,0.360441,0.383788,0.406156,0.431094,0.456412,0.485307


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0167084,-0.0298668,-0.0553163,-0.0746374,-0.0925608,-0.108949,-0.127351,-0.146278,-0.165369,-0.185188,-0.20745,-0.23033,-0.255496,-0.282115,-0.306095
0.25,0.043234,0.0272107,0.0205205,0.0166624,0.0143044,0.011971,0.0102379,0.0090407,0.00736603,0.00620867,0.00581208,0.00450793,0.00361512,0.00263499,0.00177915
0.5,0.0553456,0.0530027,0.0529735,0.0538664,0.0554516,0.0562183,0.0574206,0.0583495,0.0593186,0.0597111,0.0601048,0.0609256,0.0612435,0.0617463,0.0622037
0.75,0.0674813,0.0784836,0.0846827,0.0895948,0.0941435,0.0984463,0.101787,0.10471,0.107243,0.109188,0.111165,0.112902,0.11477,0.116223,0.117529
0.98,0.09381,0.133587,0.154746,0.171002,0.184962,0.197924,0.21139,0.225214,0.238797,0.251593,0.267291,0.277991,0.292556,0.306962,0.324723


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0481616,-0.0857382,-0.105111,-0.12137,-0.135686,-0.150027,-0.165885,-0.18316,-0.20077,-0.219684,-0.241729,-0.265108,-0.288269,-0.311736,-0.337985
0.25,-0.0223152,-0.030683,-0.0335449,-0.0358878,-0.0373826,-0.0388076,-0.0400724,-0.0415724,-0.0431537,-0.044327,-0.0449925,-0.0456991,-0.0469876,-0.0476059,-0.0479214
0.5,-0.0101964,-0.00592789,-0.00313056,-0.00105572,0.000414792,0.00152465,0.00291836,0.00332886,0.00367574,0.00460775,0.00484033,0.00532705,0.00591959,0.00607638,0.00642981
0.75,0.00169682,0.0183759,0.0263822,0.0319104,0.0362579,0.0403679,0.043273,0.0457398,0.0481579,0.0502275,0.0515754,0.0533406,0.0548185,0.0559571,0.0564203
0.98,0.0276384,0.0714435,0.0921169,0.1072,0.119076,0.132202,0.144373,0.156531,0.168707,0.17989,0.193003,0.204985,0.216475,0.230532,0.243261


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.00498865,-0.0168493,-0.0287387,-0.037657,-0.0449392,-0.049611,-0.0568062,-0.0620419,-0.0685096,-0.07485,-0.0810933,-0.0867232,-0.0927015,-0.0990773,-0.106204
0.25,0.0220882,0.0180285,0.0151265,0.0129887,0.0115938,0.0101401,0.00906979,0.00822724,0.00720201,0.00652478,0.00591372,0.00527317,0.0047882,0.00429858,0.00398928
0.5,0.029896,0.0319043,0.0314969,0.030983,0.0308086,0.0305736,0.0300637,0.0299175,0.0295675,0.0296569,0.0293814,0.0292553,0.0291492,0.0290456,0.0289106
0.75,0.0379254,0.0466873,0.0491817,0.0504423,0.0516754,0.0524143,0.0531641,0.0537984,0.054363,0.0548582,0.055292,0.0556951,0.0561294,0.056721,0.0569466
0.98,0.0563924,0.0875306,0.100764,0.110227,0.1187,0.126059,0.135282,0.143903,0.153999,0.162532,0.173153,0.184809,0.196272,0.209067,0.224069


In [30]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[4,20,40,60]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], string(assets_names[n]))    
end

,1,2,3,4
0.02,-0.137414,-0.022755,0.00127509,0.0140085
0.1,-0.0400053,0.0160845,0.0247501,0.0310375
0.25,0.0420982,0.0462731,0.0431344,0.0455853
0.5,0.141388,0.0794099,0.0651323,0.0620945
0.75,0.251677,0.114444,0.0874368,0.0786453
0.9,0.359518,0.146683,0.106862,0.0939325
0.98,0.509207,0.191123,0.133844,0.114542


,1,2,3,4
0.02,-0.169453,-0.0397351,-0.0185439,-0.0130611
0.1,-0.116099,-0.0236589,-0.00854625,-0.00516369
0.25,-0.072263,-0.0103109,-0.00102781,0.000788368
0.5,-0.0217906,0.00449305,0.00776574,0.00750977
0.75,0.0326705,0.0195271,0.0162989,0.014405
0.9,0.0839054,0.0334738,0.0243995,0.0207166
0.98,0.154702,0.0515081,0.0347305,0.0282654


,1,2,3,4
0.02,-0.0184106,-0.0115214,-0.0174087,-0.0204694
0.1,-0.00704473,-0.00331261,-0.00857073,-0.0115251
0.25,0.00233549,0.0031064,-0.00162631,-0.00453782
0.5,0.012398,0.0103206,0.00618258,0.00342495
0.75,0.0226634,0.0176406,0.0143673,0.0113517
0.9,0.0320464,0.0239761,0.0211695,0.018578
0.98,0.0439073,0.0327511,0.0302092,0.0275474


In [54]:
for n in 1:3
    s_ann = expm1.(annualise(scenarios[n,:,2:end], 12))
    print_percentiles(s_ann,[.1, .3, .5, .7, .9], string(assets_names[n]))
end


,1,2,3,4,5
0.1,0.0233337,-0.131337,-0.132156,-0.133297,-0.124587
0.3,0.186504,0.0296336,0.0331163,0.0367358,0.042474
0.5,0.313388,0.16847,0.169525,0.172049,0.173961
0.7,0.463807,0.32248,0.319701,0.328768,0.326633
0.9,0.701063,0.569683,0.571192,0.580004,0.586201


,1,2,3,4,5
0.1,-0.127215,-0.126294,-0.138864,-0.143008,-0.149985
0.3,-0.0559908,-0.0336996,-0.0425432,-0.0504487,-0.0514349
0.5,-0.00327123,0.0347976,0.0293209,0.0246577,0.0189265
0.7,0.0539829,0.113875,0.108738,0.105494,0.0987258
0.9,0.143939,0.234297,0.232656,0.23634,0.221701


,1,2,3,4,5
0.1,-0.00379308,-0.0415864,-0.0610186,-0.0738594,-0.0788032
0.3,0.0196081,-0.00606579,-0.0219584,-0.0323569,-0.0368823
0.5,0.0368322,0.0194971,0.00681871,-0.00316204,-0.00806758
0.7,0.0542163,0.04619,0.0352979,0.0283789,0.0224828
0.9,0.0789422,0.084657,0.0783586,0.071786,0.0680585


In [38]:
kurtosis(rand(Normal(0,1),1000_000))

0.006141041016670901